In [ ]:
#https://stackabuse.com/python-for-nlp-topic-modeling/

In [ ]:
import pandas as pd
import numpy as np

reviews_datasets = pd.read_csv('/content/drive/MyDrive/abcnews-date-text.csv')
reviews_datasets = reviews_datasets
reviews_datasets.dropna()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1226253,20201231,what abc readers learned from 2020 looking bac...
1226254,20201231,what are the south african and uk variants of ...
1226255,20201231,what victorias coronavirus restrictions mean f...
1226256,20201231,whats life like as an american doctor during c...


In [ ]:
reviews_datasets.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
reviews_datasets['headline_text'][350]

'offers roll in for classic holden'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(reviews_datasets['headline_text'].values.astype('U'))

In [ ]:
doc_term_matrix

<1226258x63374 sparse matrix of type '<class 'numpy.int64'>'
	with 6323014 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

In [ ]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

In [ ]:
first_topic = LDA.components_[0]

In [ ]:
top_topic_words = first_topic.argsort()[-10:]

In [ ]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

In [ ]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

In [ ]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

In [ ]:
reviews_datasets['Topic'] = topic_values.argmax(axis=1)

In [ ]:
reviews_datasets.head()

In [ ]:
str_mtr=["infectious disease appeared first time in China is killing 120 people an hour in India and it could stay really grim for months"]
abc=count_vect.transform(str_mtr)
xyz=LDA.transform(abc)
tt=xyz.argmax(axis=1)
print(tt)
